In [56]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import time
import pickle

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras import Model
from keras.optimizers import Adam
from keras.utils import to_categorical


Using TensorFlow backend.


In [3]:
import numpy as np
from mlxtend.classifier import StackingCVClassifier
from nltk import SnowballStemmer, WordNetLemmatizer, PorterStemmer
import pandas as pd
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import random
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
# from sklearn.decomposition import TruncatedSVD   # too expensive!a
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
SEED = 15
random.seed(SEED)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:
def preprocess(text):
    token = RegexpTokenizer(r'[a-zA-Z]+')
    wnl = WordNetLemmatizer()
    ps = SnowballStemmer('english')
#     ps = PorterStemmer()
    
    tokens = token.tokenize(text.lower())
    
    tokens_l = [wnl.lemmatize(token) for token in tokens]
    
    tokens_ls = [ps.stem(token) for token in tokens_l]
    
    return tokens_l # without stemming



In [0]:
# alternate preprocessing for further data cleaning(RESULTED IN REDUCTION IN PERFORMANCE)
def remove_one_letter(array):
    to_ret = []
    
    for token in array:
        if len(token) > 1:
            to_ret.append(token)

    return to_ret
            

def preprocess(text):
    token = RegexpTokenizer(r'[a-zA-Z]+')
    wnl = WordNetLemmatizer()
    ps = SnowballStemmer('english')
    text = re.sub(string=text, pattern='http[^-\s]+youtube\.[^-\s]*watch[^-\s]*', repl='')
    
    tokens = token.tokenize(text.lower())
    
    
    general_site_protocols = ['https',
                             'http',
                             'www']
    
    general_site_names = ['youtube',
                          'youtu',
                         'wikipedia',
                         'reddit',
                         'google',
                         'wiki']
    
    general_site_domains = ['com',
                           'org',
                           'net']    
    
    
    
    tokens_more_than_one_char = []
    i = 0
    for i in range(len(tokens)):
        token = tokens[i]
        if len(token) > 1 and not ((token in general_site_protocols) \
                                   or (token in general_site_names) \
                                   or (token in general_site_domains)):
            tokens_more_than_one_char.append(token)

    tokens_l = [wnl.lemmatize(token) for token in tokens_more_than_one_char]
    
    return tokens_l # without stemming




DATA LOADING+ PREPROCESSING

In [0]:

train = pd.read_csv('/content/drive/My Drive/reddit-comment-classification-comp-551/reddit_train.csv')
test = pd.read_csv('/content/drive/My Drive/reddit-comment-classification-comp-551/reddit_test.csv')



In [0]:
y_labels_train = train['subreddits'].values
x_comments_train = train['comments'].values
x_comments_test = test['comments'].values

In [0]:
# train

In [0]:
labels = np.unique(y_labels_train)
labels_to_index = {w: int(np.where(labels==w)[0]) for w in labels}
index_to_labels={v:k for k,v in labels_to_index.items()}
train['labels'] = train['subreddits'].map(labels_to_index)

In [0]:
y = train['labels'].values
# y_train

In [11]:
del train['id']
del test['id']

train=train.rename(columns={"comments": "text", "subreddits": "label"})
test=test.rename(columns={"comments": "text"})
cols=train.columns.tolist()
print(train.head())
print(cols)
train=train[["label","text","labels"]]
print(train.head())

                                                text            label  labels
0  Honestly, Buffalo is the correct answer. I rem...           hockey      11
1  Ah yes way could have been :( remember when he...              nba      14
2  https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...  leagueoflegends      12
3  He wouldn't have been a bad signing if we woul...           soccer      16
4  Easy. You use the piss and dry technique. Let ...            funny       9
['text', 'label', 'labels']
             label                                               text  labels
0           hockey  Honestly, Buffalo is the correct answer. I rem...      11
1              nba  Ah yes way could have been :( remember when he...      14
2  leagueoflegends  https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...      12
3           soccer  He wouldn't have been a bad signing if we woul...      16
4            funny  Easy. You use the piss and dry technique. Let ...       9


In [0]:
train['text'] = train['text'].str.replace("[^a-zA-Z]", " ")
test['text'] = test['text'].str.replace("[^a-zA-Z]", " ")

In [0]:
tokenized_doc = train['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item.lower() for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(train)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

train['text'] = detokenized_doc

In [0]:
tokenized_doc = test['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item.lower() for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(test)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

test['text'] = detokenized_doc

In [46]:
df_trn, df_val = train_test_split(train,test_size = 0.1, random_state=SEED,shuffle=False)
x_train=df_trn['text'].values
y_train=df_trn['labels'].values
x_val=df_val['text'].values
y_val=df_val['labels'].values
df_trn=df_trn.reset_index(drop = True)
df_val=df_val.reset_index(drop = True)
print(len(df_val))
sample=int(0.5*len(df_val))
print(sample)
df_val1=df_val[:sample]
df_val2=df_val[sample:]
print(len(df_val2),len(df_val1))

7000
3500
3500 3500


In [52]:
print(df_val2.head())
# del df_val2['index']
df_val2=df_val2.reset_index(drop=True)
df_val2.head()

                label  ... labels
8750           canada  ...      6
8751        AskReddit  ...      0
8752  leagueoflegends  ...     12
8753       conspiracy  ...      7
8754           hockey  ...     11

[5 rows x 3 columns]


,label,text,labels
0,canada,gt this purchase benefit ontarians way it done...,6
1,AskReddit,they come color cows either answer correct,0
2,leagueoflegends,https twitter com tsmdoublelift status steve g...,12
3,conspiracy,at point i think bot intelligent offense war c...,7
4,hockey,oh i see thanks dissecting frog clue worked be...,11


ULMFiT language model fine tuning+ classifier training

In [0]:
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val,text_cols='text',label_cols='label',path = "",min_freq=2)

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, test_df=test,text_cols='text',label_cols='label',bs=32)

In [48]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.9)
learn.fit_one_cycle(cyc_len=1, max_lr=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.195064,6.818326,0.116551,03:40


In [49]:
learn.unfreeze()
learn.fit_one_cycle(cyc_len=3, max_lr=2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.628860,6.493684,0.139868,04:17
1,6.434684,6.354203,0.150131,04:16
2,6.128782,6.325001,0.153109,04:16


In [0]:
learn.save_encoder('ft_enc')

In [51]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.9)
learn.load_encoder('ft_enc')

learn.freeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.990416,1.639660,0.500571,02:41


In [52]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.976006,1.599307,0.513857,03:23


In [53]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.815203,1.542576,0.532286,04:32


In [54]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3))


epoch,train_loss,valid_loss,accuracy,time
0,1.851728,1.518229,0.535571,06:40
1,1.781112,1.514887,0.543571,07:06
2,1.732981,1.490845,0.549714,07:41
3,1.654869,1.464984,0.557000,07:06
4,1.627550,1.445955,0.566000,06:35
5,1.536946,1.437045,0.565286,06:54
6,1.505937,1.433024,0.566571,07:14
7,1.505341,1.422740,0.569571,06:57


Buffered data was truncated after reaching the output size limit.

In [0]:
val_predsulm, _ = learn.get_preds(DatasetType.Valid,ordered=True)
val2_predsulm, _ = learn.get_preds(DatasetType.Test,ordered=True)
val1_predsulm=val1_predsulm.numpy()
val2_predsulm=val2_predsulm.numpy()

In [0]:
test_predsulm, _ = learn.get_preds(DatasetType.Test,ordered=True)
test_predsulm=test_predsulm.numpy()

In [0]:
#defining tf-idf vectorizer
cv_unigram_tfidf = TfidfVectorizer(lowercase=True,
                     stop_words=stop_words,
                     analyzer='word',
                     ngram_range = (1,1),
                     tokenizer = preprocess,
                    binary=False)


In [31]:
sample=int(0.5*len(x_val))


8750


In [0]:
x_val1=x_val[:sample]
x_val2=x_val[sample:]
y_val1=y_val[:sample]
y_val2=y_val[sample:]

In [33]:
#training and validation data
x_train_unigram_tfidf = cv_unigram_tfidf.fit_transform(x_train)
x_val_unigram_tfidf = cv_unigram_tfidf.transform(x_val)
x_val1_unigram_tfidf = cv_unigram_tfidf.transform(x_val1)
x_val2_unigram_tfidf = cv_unigram_tfidf.transform(x_val2)
x_test_unigram_tfidf=cv_unigram_tfidf.transform(x_comments_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
optim=Adam(lr=0.0001)

In [73]:
knn=KNeighborsClassifier(n_neighbors=250,algorithm='auto',weights='distance')
t1=time.time()
knn.fit(x_train_unigram_tfidf1, y)
time.time()-t1
# x=knn.predict_proba(x_train_unigram_tfidf)
# y=knn.predict_proba(x_val_unigram_tfidf)
# knn.fit(x_train_unigram_tfidf, y_train)
# print(knn.score(x_val_unigram_tfidf, y_val))

0.020282745361328125

In [59]:
#creating final training data.
x_train_unigram_tfidf1 = cv_unigram_tfidf.fit_transform(x_comments_train)
x_test_unigram_tfidf=cv_unigram_tfidf.transform(x_comments_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
#SVM initial training
sg=SGDClassifier(loss='hinge',alpha=1e-4,max_iter=1000)
csg = CalibratedClassifierCV(base_estimator=sg, method='isotonic', cv=3)
csg.fit(x_train_unigram_tfidf,y_train)
svtrpred=csg.predict_proba(x_train_unigram_tfidf)
svvalpred=csg.predict_proba(x_val_unigram_tfidf)
svval1pred=csg.predict_proba(x_val1_unigram_tfidf)
svval2pred=csg.predict_proba(x_val2_unigram_tfidf)

# print(csg.score(x_val_unigram_tfidf, y_val))

In [63]:
#svm final training on whole dataset
sg=SGDClassifier(loss='hinge',alpha=1e-4,max_iter=1000)
csg = CalibratedClassifierCV(base_estimator=sg, method='isotonic', cv=3)
t1=time.time()
csg.fit(x_train_unigram_tfidf1,y)
print(time.time()-t1)
svtestpred=csg.predict_proba(x_test_unigram_tfidf)
# print(csg.score(x_val_unigram_tfidf, y_val))

5.38595724105835


In [36]:
#logistic regression initial training
lr1 = LogisticRegression(max_iter=300,C=1)
lr1.fit(x_train_unigram_tfidf, y_train)
lrtrpred=lr1.predict_proba(x_train_unigram_tfidf)
lrvalpred=lr1.predict_proba(x_val_unigram_tfidf)
lrval1pred=lr1.predict_proba(x_val1_unigram_tfidf)
lrval2pred=lr1.predict_proba(x_val2_unigram_tfidf)

print(lr1.score(x_val_unigram_tfidf, y_val))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [64]:
#final logistic regression on whole data
lr1 = LogisticRegression(max_iter=300,C=1)
t1=time.time()
lr1.fit(x_train_unigram_tfidf1, y)
print(time.time()-t1)
lrtestpred=lr1.predict_proba(x_test_unigram_tfidf)
# print(lr1.score(x_val_unigram_tfidf, y_val))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


18.42897653579712


In [0]:
# initial naive bayes training

nb3 = MultinomialNB(alpha=0.18) 
nb3.fit(x_train_unigram_tfidf, y_train)
nbtrpred=nb3.predict_proba(x_train_unigram_tfidf)
nbvalpred=nb3.predict_proba(x_val_unigram_tfidf)
nbval1pred=nb3.predict_proba(x_val1_unigram_tfidf)
nbval2pred=nb3.predict_proba(x_val2_unigram_tfidf)
# print(nb3.score(x_val_unigram_tfidf, y_val))

In [65]:
# final naive bayes training on whole dataset

nb3 = MultinomialNB(alpha=0.18)
t1=time.time()
nb3.fit(x_train_unigram_tfidf1, y)
print(time.time()-t1)
nbtestpred=nb3.predict_proba(x_test_unigram_tfidf)
# print(nb3.score(x_val_unigram_tfidf, y_val))

0.1432945728302002


In [0]:
#concatenation of validation split predictions for training meta classifier
pred_val=np.concatenate((svvalpred,lrvalpred,nbvalpred,val_predsum),axis=1)
pred_val1=np.concatenate((svval1pred,lrval1pred,nbval1pred,val1_predsulm),axis=1)
pred_val2=np.concatenate((svval2pred,lrval2pred,nbval2pred,val2_predsulm),axis=1)

In [0]:
#concatenation of test data probabilities predictions for training meta classifier
predtest=np.concatenate((svtestpred,lrtestpred,nbtestpred,test_predsulm),axis=1)


In [0]:
pkl_filename = "/content/drive/My Drive/reddit-comment-classification-comp-551/lr_model.pkl"

In [0]:
#training and testing meta classifier on the split validation of stacked model on data split in ratio 75-25%
lr1=LogisticRegression()
lr1.fit(pred_val1,y_val1)
lr1.score(pred_val2,y_val2)

In [42]:
#final training of meta classifier and saving it for later
lr=LogisticRegression()
lr.fit(pred_val,y_val)
# print(lr.score(pred_val2,y_val2))

with open(pkl_filename, 'wb') as file:
    pickle.dump(lr, file)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5954285714285714


In [0]:
with open(pkl_filename, 'rb') as file:
    lr = pickle.load(file)
preds=lr.predict(predtest)

In [28]:
#ensemble classfier without ULMFiT
# for voting dont fit the naive bayes and logistic only define them
eclf1 = VotingClassifier(estimators=[('lr1', lr1), ('nb', nb3),('sg',csg)], voting='soft')
eclf1 = eclf1.fit(x_train_unigram_tfidf, y_train)
print(eclf1.score(x_val_unigram_tfidf, y_val))
# print(eclf1.score(x_train_unigram_tfidf, y_train))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5780571428571428


In [0]:
#final prediction using ensemble classifier
eclf1 = VotingClassifier(estimators=[
...         ('lr', lr1), ('nb', nb3),('sg',csg)], voting='soft')
eclf1 = eclf1.fit(x_train_unigram_tfidf1, y)
# print(eclf1.score(x_val_unigram_tfidf, y_val))
# print(eclf1.score(x_train_unigram_tfidf1, y))
preds=eclf1.predict(x_test_unigram_tfidf)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8226


In [0]:
#making csv for kaggle test data
ids=[]
labels=[]

for i in range(preds.shape[0]):
    ids.append(i)
    labels.append(index_to_labels[preds[i]])
out_df=pd.DataFrame({'Id':ids,
                    'Category':labels})
order=['Id','Category']
out_df[order].to_csv('/content/drive/My Drive/reddit-comment-classification-comp-551/test3.csv',index=False)